# Recommender systems

## One of the most common uses of big data is to predict and suggest what users may want.  This allows Google to show you relevant ads or to suggest news in Google Now; Amazon to recommend relevant products; Netflix to recommend movies that you might like; or most recently, the famous **Weekly Dicovery** of Spotify.

## All these products are based on systems of recommendation: a information retrieval method to provide users with relevant, yet novel and diverse, information. 

## In this class we will use a pretty famous dataset based on movies ratings, 'MovieLens', to learn the basics of recommender systems. 

## Table of Contents (times are approximated)

1. [Getting and analysing some data (1.5-2 h, typically until break)](#data)
2. [Most popular movies (0.5-1 min)](#popular)
3. [Metrics for recommender systems (2-2.5h)](#metrics)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import io
import os

<a id='data'></a>
## 1.1 Load data

We will use MovieLens dataset, which is one of the most common datasets used when implementing and testing recommender engines. This data set consists of:
* 100,000 ratings (1-5) from 943 users on 1682 movies. 
* Each user has rated at least 20 movies. 
* Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens [website](https://movielens.org) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set.

You can download the dataset [here](http://files.grouplens.org/datasets/movielens/ml-100k.zip).

Take a look at the readme file!!!

In [2]:
data_root = "../data/ml-100k/"

In [3]:
!cat {data_root}README

SUMMARY & USAGE LICENSE

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
	* 100,000 ratings (1-5) from 943 users on 1682 movies. 
	* Each user has rated at least 20 movies. 
        * Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set. Detailed descriptions of
the data file can be found at the end of this file.

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under th

In [4]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']
datafile = os.path.join(data_root, "u.data")
data = pd.read_csv(datafile, sep='\t', names=columns)
data.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


### A remainder of the numpy library

*Pandas library is nothing alse than numpy under the hood (numpy with steroids, if you like). You can access the data (in matrix from) with he "values" attribute, e.g. data.values*

In [6]:
data.values

array([[      196,       242,         3, 881250949],
       [      186,       302,         3, 891717742],
       [       22,       377,         1, 878887116],
       ...,
       [      276,      1090,         1, 874795795],
       [       13,       225,         2, 882399156],
       [       12,       203,         3, 879959583]])

In [7]:
# The attribute shape provides the shape of the matrix
data.values.shape

(100000, 4)

In [9]:
# Note that if we return the first column, we get a vector (of 100000 components)
data.values[:, 0].shape

(100000,)

In [11]:
# same with the first row (this time, we get a vector of 4 components)
data.values[0, :]
data.values[0]

array([      196,       242,         3, 881250949])

In [12]:
# access all rows, and first 3 columns 
data.values[:, :3]

array([[ 196,  242,    3],
       [ 186,  302,    3],
       [  22,  377,    1],
       ...,
       [ 276, 1090,    1],
       [  13,  225,    2],
       [  12,  203,    3]])

In [ ]:
# access all collumns, and first 3 rows 


In [ ]:
# access first 10 rows

# This is equivalent to data.values[:10]

In [ ]:
# access first column


In [14]:
# Number of users and items
n_users = data.user_id.unique().size

In [16]:
# Number of items
n_items = np.unique(data.values[:, 1]).shape[0]

In [17]:
print("There are %s users and %s items" %(n_users, n_items))

There are 943 users and 1682 items


## 1.2 A dictionary for movies and a search tool

In order to analyze the predicted recommendations, let's create a python dictonary that will allow us to translate any item id to the corresponding movie title. Also, let's write a small function that returns the ids of the movies containing some text.

The correspondance between titles and ids is stored in the u.item file

In [18]:
# data_root = "./../data/ml-100k/"
items_id_file = os.path.join(data_root, "u.item")
!head $items_id_file

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0
6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0
8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Babe%20(1995)|0|0|0|0|1

### *Simple reminder of dictionaries*

In [19]:
aux = {'hola': 'que haces?', 1: '237'}

In [20]:
# Access value of key='hola'
aux['hola']

'que haces?'

In [22]:
# create new key
aux['adios'] = 'ciao'
aux

{'hola': 'que haces?', 1: '237', 'adios': 'ciao'}

In [23]:
# Update value of existing key
aux['hola'] = 'hola'
aux

{'hola': 'hola', 1: '237', 'adios': 'ciao'}

In [25]:
# get all keys 
list(aux.keys())

['hola', 1, 'adios']

In [26]:
# get all values
aux.values()

dict_values(['hola', '237', 'ciao'])

In [27]:
# get all keys and values at the same time
aux.items()

dict_items([('hola', 'hola'), (1, '237'), ('adios', 'ciao')])

In [28]:
# Create a dictionary for movie titles and ids
item_dict = {}
with io.open(items_id_file, 'rb') as f:
    for line in f.readlines():
        record = line.split(b'|')
        item_dict[int(record[0])] = str(record[1])
    
# We can use this dict to see the films a user has seen, for instance. 
for record in data.values[:20]:
    print("User {u} viewed '{m}' and gave a {r} rating".format(
        u=record[0], m=item_dict[record[1]], r=record[2]))    

User 196 viewed 'b'Kolya (1996)'' and gave a 3 rating
User 186 viewed 'b'L.A. Confidential (1997)'' and gave a 3 rating
User 22 viewed 'b'Heavyweights (1994)'' and gave a 1 rating
User 244 viewed 'b'Legends of the Fall (1994)'' and gave a 2 rating
User 166 viewed 'b'Jackie Brown (1997)'' and gave a 1 rating
User 298 viewed 'b'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)'' and gave a 4 rating
User 115 viewed 'b'Hunt for Red October, The (1990)'' and gave a 2 rating
User 253 viewed 'b'Jungle Book, The (1994)'' and gave a 5 rating
User 305 viewed 'b'Grease (1978)'' and gave a 3 rating
User 6 viewed 'b'Remains of the Day, The (1993)'' and gave a 3 rating
User 62 viewed 'b'Men in Black (1997)'' and gave a 2 rating
User 286 viewed 'b"Romy and Michele's High School Reunion (1997)"' and gave a 5 rating
User 200 viewed 'b'Star Trek: First Contact (1996)'' and gave a 5 rating
User 210 viewed 'b'To Wong Foo, Thanks for Everything! Julie Newmar (1995)'' and gave a 3 

In [32]:
# Define a function that retrieves all the ids and titles for movies containing 'text' in its title
def returnItemId(text, ids):
    """
    :param text: string to be looked for in movies titles
    :param ids: dictionary of {id:title}
    
    :return: a list of (id,title) if text found in titles, and an empty list otherwise.
    """
    # convert input text to lowercase
    text_ = text.lower()
    # find occurances
    search = []
    for id_,title in ids.items():
        if text in title.lower():
            search.append((id_, title))
    
    return search

In [33]:
returnItemId??

In [34]:
returnItemId('but', item_dict)

[(240, "b'Beavis and Butt-head Do America (1996)'"),
 (435, "b'Butch Cassidy and the Sundance Kid (1969)'"),
 (580,
  "b'Englishman Who Went Up a Hill, But Came Down a Mountain, The (1995)'"),
 (1401, "b'M. Butterfly (1993)'"),
 (1459, "b'Madame Butterfly (1995)'"),
 (1614, "b'Reluctant Debutante, The (1958)'"),
 (1621, "b'Butterfly Kiss (1995)'"),
 (1645, "b'Butcher Boy, The (1998)'"),
 (1650, "b'Butcher Boy, The (1998)'")]

## 1.3 Data consistency (always double check everything!!!)

In [37]:
# check whether titles are unique or not. They are not!!!
len(set(item_dict.keys())), len(set(item_dict.values()))

(1682, 1664)

In [ ]:
?

### One work around: create another dict that consolidates ids with the same movie title

Alternatively, you could create a Dataframe from the above dictionary of ids:titles

<pre><code>
dd = pd.DataFrame.from_dict(item_dict, orient='index').reset_index()
dd.columns = ['item_id', 'title']
dd.head()
</code></pre>

find the duplicates (`dd.groupby('title')...`), and create a new id for the items. 

In [ ]:
duplicates_item_dict = {}
# Las claves en "duplicates_item_dict" son los nombres de las películas
# Los valores son una lista de los ids (que pueden ser uno solo, o varios)
for id_, name in list(item_dict.items()):
    # clave: name; valor: list(id_)
    ?

# show the duplicated titles
for k,v in list(duplicates_item_dict.items()):
    if len(v)>1:
        print(k,v)

Create a dict where the key are the original ids, and the values are the unique one. 
We will use this dictionary to remove duplicates in a dataframe.

In [ ]:
unique_id_item_dict ={}
for new_id, old_id_list in enumerate(duplicates_item_dict.values()):
    # key: old_id; value: new_id
    ?

unique_id_item_dict = {old_id: new_id for new_id, old_id_list in enumerate(duplicates_item_dict.values()) 
                                      for old_id in old_id_list}

In [ ]:
unique_id_item_dict

Create another dict mapping moving titles to this new unique id

In [ ]:
unique_item_dict = {unique_id_item_dict[k]:v 
                    for k,v in item_dict.items()}
    
assert(len(set(unique_item_dict.keys())) == 
       len(set(unique_item_dict.values())))

Now we can use our `returnItemId()` mehtod safely =)

In [ ]:
returnItemId('but', unique_item_dict)

In [ ]:
returnItemId('but', item_dict)

## Copiamos de notebook de mañana 

In [49]:
class rueda():
    
    def __init__(self, marca, diametro):
        self.marca = marca
        self.diametro = diametro
        self.distancia = 0
        
    def rodar(self, distancia):
        self.distancia += distancia
        

In [50]:
r1 = rueda(marca='michelin', diametro=17)
r2 = rueda(marca='michelin', diametro=42)

In [51]:
r1.diametro, r2.diametro

(17, 42)

In [52]:
for _ in range(10):
    r1.rodar(5)
    print(r1.distancia)

5
10
15
20
25
30
35
40
45
50


In [53]:
class movielens_100k(object):
    """
    This python class read the movielens-100k dataset and:
        . Preprocess train and test sets
        . Create a python dict that allows us to translate any item id to the corresponding movie title
    """
    
    def __init__(self, data_root, item_id_file='u.item', train_file='u2.base', test_file='u2.test', 
                 columns=['user_id', 'item_id', 'rating', 'timestamp']):
                
        self.data_root = data_root
        self.items_id_file = os.path.join(self.data_root, item_id_file)
        self.train_file = train_file
        self.test_file = test_file
        self.columns = columns
        self.__data = {}  # dictionary to store train and test data
            
    def __getitem__(self, dataset):
        """
        Get train and test data

        :param dataset: dataset name (String)
        :return: pandas dataframe
        """
        return self.__data[dataset]

    def __setitem__(self, key, value):
        """
        Store train and test data in the dictionary self.__data.

        :param key: name of the dataset (String)
        :param value: pandas dataframe
        """
        self.__data[key] = value
        
    def read_data(self, file, verbose=False):
        """
        Read file returning a pandas dataframe
        :param file: file path
        :param verbose: print the first rows of the data
        """
        datafile = os.path.join(self.data_root, file)
        data = pd.read_csv(datafile, sep='\t', names=self.columns)
        if verbose:
            print(data.head())
        return data
            
    def returnItemId(self, text, dict_ids):
        """
        Retrieves all the ids and titles for movies containing 'text' in its title
        :param text: string to be looked for in movies titles
        :param dict_ids: dicttionary of {id:title}

        :return: a list of (id,title) if text found in titles, and an empty list otherwise.
        """
        # convert input text to lowercase
        text_ = text.lower()
        # find occurances
        search = [(k, v.lower().find(text_)) 
                  for k,v in list(dict_ids.items())]

        # Get the IDs corresponding to the given text
        index = [k for k,v in search if v>-1]

        # Return a list with the id and the name
        out = []
        for i in index:
            out.append((i, dict_ids[i]))
        return out
    
    def check_data_consistency(self):
        """
        Check if one film has two or more ids
        """
        self.item_dict = {}
        with io.open(self.items_id_file, 'rb') as f:
            for line in f.readlines():
                record = line.split(b'|')
                self.item_dict[int(record[0])] = str(record[1])
                
        if len(set(self.item_dict.keys())) == len(set(self.item_dict.values())):
            self.unique_item_dict_ = self.item_dict
            return True
        else:
            self.duplicates_item_dict = {}
            for id_,name in list(self.item_dict.items()):
                if name not in self.duplicates_item_dict:
                    self.duplicates_item_dict[name] = [id_]
                else:
                    self.duplicates_item_dict[name] = self.duplicates_item_dict[name]+[id_]

            self.unique_id_item_dict ={}
            for i, lista_id in enumerate(self.duplicates_item_dict.values()) :
                for key in lista_id:
                    self.unique_id_item_dict[key] = i

            self.unique_item_dict_ = {self.unique_id_item_dict[k]:v 
                                for k,v in self.item_dict.items()}
            return False
    
    def correcting_non_unique_movies(self, data_df):
        """
        Preprocess data_df in case the check_data_consistency function fails
        """
        if not self.check_data_consistency():
            data_df['item_id'] = data_df['item_id'].apply(lambda x: self.unique_id_item_dict[x])
            
        return data_df
            
    def train(self, verbose=False):
        """
        Train data
        """
        self['train_set'] = self.read_data(self.train_file, verbose)
        self['train_set'] = self.correcting_non_unique_movies(self['train_set'])
        items = self['train_set'].item_id.unique()
        self.train_id_dict = {j:i for i,j in enumerate(items)}
        self['train_set']['item_id'] = self['train_set']['item_id'].apply(lambda x: self.train_id_dict[x])
        self.unique_item_dict = {self.train_id_dict[v]:self.unique_item_dict_[v] 
                                for v in items}
        self.n_users = self['train_set'].user_id.unique().shape[0]
        self.n_items = self['train_set'].item_id.unique().shape[0]
        self.n_pairs = self['train_set'].shape[0]
        if verbose:
            print('There are %s users, %s items and %s pairs in the test set' \
                  %(self.n_users, self.n_items, self.n_pairs))
    
    def test(self, verbose=False):
        """
        Test data
        """
        self['test_set'] = self.read_data(self.test_file, verbose)
        self['test_set']['item_id'] = self['test_set']['item_id'].apply(lambda x: self.unique_id_item_dict[x])
        self['test_set']['item_id'] = self['test_set']['item_id'].apply(lambda x: self.train_id_dict[x] if x in self.train_id_dict else -1)
        self['test_set'] = self['test_set'][self['test_set']['item_id']!=-1]
        if verbose:
            print('There are %s users, %s items and %s pairs in the test set' \
                  %(self['test_set'].user_id.unique().shape[0], self['test_set'].item_id.unique().shape[0], 
                    self['test_set'].shape[0]))
            
    def get_data(self, verbose=False):
        """
        A kind of main function to return train and test data together
        """
        self.train(verbose)
        self.test(verbose)

In [57]:
m1 = movielens_100k(data_root=data_root)
m1.get_data()

In [59]:
m1['train_set'].head()

,user_id,item_id,rating,timestamp
0,1,0,4,878542960
1,1,1,3,876893119
2,1,2,3,889751712
3,1,3,5,887431973
4,1,4,4,875071561


In [60]:
m1['test_set'].head()

,user_id,item_id,rating,timestamp
0,1,289,5,874965758
1,1,290,3,876893171
2,1,373,1,875072484
3,1,374,5,878543541
4,1,665,1,878542772


In [61]:
m2 = movielens_100k(data_root=data_root, train_file='u1.base', test_file='u1.test')
m2.get_data()
m2['test_set'].head()

,user_id,item_id,rating,timestamp
0,1,740,5,887431973
1,1,134,3,875693118
2,1,268,5,878542960
3,1,135,5,874965706
4,1,536,3,875073198


## 1.4 Train and test sets

GroupLens provides several splits of the dataset, so that we can check the goodness of our algorithms. See the README file for more  details. Here we will use one of such splits.

Please notice that we have to correct for the non-unique movie's id issue!!

In [ ]:
l!ls $data_root

In [ ]:
trainfile = os.path.join(data_root, 'ua.base')
!head $trainfile

In [ ]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']
trainfile = os.path.join(data_root, "ua.base")
train = ?
print('There are %s users, %s itmes and %s pairs in the train set' \
      %(train.user_id.unique().shape[0], train.item_id.unique().shape[0], train.item_id.count()))
train.head()


In [ ]:
# same for test
columns = ['user_id', 'item_id', 'rating', 'timestamp']
testfile = os.path.join(data_root, "ua.test")
test = ?
print('There are %s users, %s itmes and %s pairs in the test set' \
      %(test.user_id.unique().shape[0], test.item_id.unique().shape[0], test.item_id.count()))
test.head()


### Correcting for non-unique movies id 

Use Pandas `apply` method. Note it can be applied to all rows in a column, thus returning a Pandas Series; or it can be applied row-wise, he

*Reminder of lambda functions in Python: is a way of calling short functions (1 line of code), without having to define the function in a separted cell.*

In [ ]:
train['item_id'] = ?

In [ ]:
test['item_id'] = ?

<a id='popular'></a>
## 2. Most popular movies

Recommending popular items is a simple, yet quite effective baseline for recommendation. Indeed, most RS suffer from a strong *popularity bias*, i.e. they tend to recommend popular items more frequently than they should -just because suggesting what is popular is effective!-. There is a lot of research  devote to understand this behaviour and to develop recipies to avoid it. 

Movies can be ranked according to different popularity metrics:
* Most rated movie (it is assumed that this is the most watched movie)
* Most positively rated movie (rating > 4.0)
* Highest rated movie

## 2.1 Most rated movie

In [62]:
train = m1['train_set']
test = m1['test_set']

In [63]:
# group the train dataset by item and count the number of users using Pandas
mostRated = train.groupby('item_id')['user_id'].count()
mostRated.head()

item_id
0     65
1    172
2     66
3     22
4    318
Name: user_id, dtype: int64

In [ ]:
mostRated.head()

In [64]:
# sort in descending order
mostRatedSorted = mostRated.sort_values(ascending=False)

In [65]:
mostRatedSorted.head()

item_id
36     461
80     414
207    409
298    406
251    402
Name: user_id, dtype: int64

In [66]:
m1.unique_item_dict

{0: "b'Four Rooms (1995)'",
 1: "b'Get Shorty (1995)'",
 2: "b'Copycat (1995)'",
 3: "b'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)'",
 4: "b'Twelve Monkeys (1995)'",
 5: "b'Richard III (1995)'",
 6: "b'Seven (Se7en) (1995)'",
 7: "b'Usual Suspects, The (1995)'",
 8: "b'Mighty Aphrodite (1995)'",
 9: "b'Postino, Il (1994)'",
 10: 'b"Mr. Holland\'s Opus (1995)"',
 11: "b'French Twist (Gazon maudit) (1995)'",
 12: "b'From Dusk Till Dawn (1996)'",
 13: "b'White Balloon, The (1995)'",
 14: 'b"Antonia\'s Line (1995)"',
 15: "b'Angels and Insects (1995)'",
 16: "b'Taxi Driver (1976)'",
 17: "b'Rumble in the Bronx (1995)'",
 18: "b'Birdcage, The (1996)'",
 19: "b'Bad Boys (1995)'",
 20: "b'Apollo 13 (1995)'",
 21: "b'Batman Forever (1995)'",
 22: "b'Crimson Tide (1995)'",
 23: "b'Desperado (1995)'",
 24: "b'Free Willy 2: The Adventure Home (1995)'",
 25: "b'Mad Love (1995)'",
 26: "b'Strange Days (1995)'",
 27: "b'Billy Madison (1995)'",
 28: "b'Clerks (1994)'",
 29: "b'Disclosure (1

In [68]:
# add the movie title, using the apply method, returning a Pandas Series in each row
def add_tile(row):
    id_ = row[0]
    title_ = m1.unique_item_dict[id_]
    freq_ = row[1]
    new_row = [id_, title_, freq_]
    return pd.Series(new_row, index=['id', 'title', 'freq'])

mostRatedMovies = (mostRatedSorted
                   .reset_index()
                   .apply(add_tile, axis=1)
                  )

In [69]:
mostRatedMovies.head()

,id,title,freq
0,36,b'Star Wars (1977)',461
1,80,b'Fargo (1996)',414
2,207,b'Contact (1997)',409
3,298,b'Return of the Jedi (1983)',406
4,251,b'Liar Liar (1997)',402


## 2.2 Most positively rated movie

Use the query method to filter values in train, `train.query('rating>4')`, or the traditional form `train[train.rating>4]`.

In [101]:
mostPositiveRatedMovies = (train
    .query('rating>=4')             
    .groupby('item_id')['user_id']
    .count()
    .sort_values(ascending=False)
    .reset_index()
    .apply(add_tile, axis=1)
)

mostPositiveRatedMovies.head()

,id,title,freq
0,36,b'Star Wars (1977)',397
1,80,b'Fargo (1996)',334
2,298,b'Return of the Jedi (1983)',302
3,105,"b'Godfather, The (1972)'",283
4,140,b'Raiders of the Lost Ark (1981)',280


## 2.3 Highest mean rating movie

In [71]:
train.groupby('item_id')['rating'].agg(list)

item_id
0       [4, 3, 3, 2, 4, 1, 3, 5, 3, 3, 3, 2, 2, 4, 4, ...
1       [3, 5, 4, 5, 4, 5, 4, 2, 4, 4, 3, 3, 3, 5, 4, ...
2       [3, 3, 4, 4, 4, 3, 3, 2, 3, 3, 3, 3, 2, 3, 4, ...
3       [5, 5, 3, 3, 5, 4, 4, 1, 4, 5, 2, 5, 4, 5, 3, ...
4       [4, 2, 5, 3, 4, 4, 5, 1, 5, 4, 4, 4, 3, 5, 4, ...
5       [3, 4, 3, 4, 4, 5, 5, 5, 5, 5, 4, 2, 5, 3, 3, ...
6       [2, 4, 3, 3, 2, 1, 2, 4, 5, 3, 3, 4, 4, 4, 5, ...
7       [5, 5, 5, 2, 5, 5, 4, 5, 4, 5, 4, 5, 4, 5, 5, ...
8       [5, 4, 3, 4, 3, 5, 4, 4, 3, 3, 5, 4, 4, 4, 4, ...
9       [5, 5, 4, 3, 5, 2, 5, 4, 4, 4, 4, 5, 4, 5, 4, ...
10      [5, 3, 5, 5, 4, 5, 4, 4, 4, 4, 5, 4, 4, 2, 5, ...
11      [5, 4, 4, 1, 3, 4, 3, 3, 3, 2, 3, 2, 4, 4, 3, ...
12      [3, 4, 4, 3, 2, 3, 4, 2, 4, 3, 5, 3, 3, 4, 2, ...
13                                  [4, 3, 1, 4, 2, 2, 3]
14      [5, 3, 4, 5, 3, 4, 4, 4, 4, 5, 5, 5, 4, 4, 4, ...
15      [4, 3, 1, 4, 3, 3, 1, 5, 2, 4, 1, 4, 4, 4, 5, ...
16      [4, 4, 5, 5, 4, 4, 5, 4, 4, 5, 4, 4, 5, 5, 5, ...
17    

In [100]:
# obtain the highest rated movies, with a minium number of users/ratings.
min_ratings = 200
highestMeanRatedMovies = (train    
    .groupby('item_id')['rating'].agg(list)
    .apply(lambda l: np.mean(l) if len(l)>=min_ratings else 0.0)
    .sort_values(ascending=False)
    .reset_index()
    .apply(add_tile, axis=1)
)

highestMeanRatedMovies.head()

,id,title,freq
0,252.0,"b""Schindler's List (1993)""",4.512821
1,49.0,"b'Shawshank Redemption, The (1994)'",4.466667
2,7.0,"b'Usual Suspects, The (1995)'",4.395122
3,36.0,b'Star Wars (1977)',4.364425
4,105.0,"b'Godfather, The (1972)'",4.301205


<div class  = "alert alert-info"> 
** QUESTION **: set the value of *min_ratings* to 1, and re-run the cell. What happens now? Change this value
</div>

<div class  = "alert alert-info"> 
** QUESTION **: Which method is better?? How to measure a recommender system? 
</div>

<div class  = "alert alert-info"> 
** IMPORTANT QUESTION **: When might be useful to recommend popular items?
</div>

<a id='metrics'></a>
## 3. Metrics for recommender systems

As we have seen, even with the simplest solution --aka, recommending popular items-- is difficult to known which technique performs better. For this, there are a number of metrics that allow one to measure the goodness of a recommender system. 

Metrics can be design for measuring the relevance or accuracy of a recommendation, but they can be created for evaluating the novelty of a recommendation, or its diversity. 

For now, we will focus on relevance and accuracy. Several metrics exist:
* Accuracy: rmse, mae.
* Not ranked: Recall@k, Precision@k.
* With rank disccount: map@k, ndcg@k.
* With rank ordering: mean percentile rank.

We will be definiing some of them whitin this class. For the moment, let's talk about precision and recall.

## 3.1 Precision and recall

<img src="https://upload.wikimedia.org/wikipedia/commons/2/26/Precisionrecall.svg" alt="Precision and Recall in IR" style="float: right; width: 300px"/>

The concept of precision and recall comes form the world of information retrieval, have a look at the wikipedia:

https://en.wikipedia.org/wiki/Precision_and_recall

From this entry:

 * "**precision** (also called positive predictive value) is the fraction of retrieved instances that are relevant".
 * "**recall** (also known as sensitivity) is the fraction of relevant instances that are retrieved".

<br />
<div class  = "alert alert-info"> 
** QUESTION **: how do we know if some movie, unknown to the user, is relevant?
</div>

In other words, we cannot measure a false positive --something recommended that was not relevant--. In this regard, only recall-oriented metrics have an actual meaning in RS. Nonetheless, its common practice to define both metrics in RS as follows:
 
### $$\mathrm{recall}@N = \frac{\sum_{k=1}^N rel(k)}{\sum_{i\in \mathcal{I}_u} 1}$$
### $$\mathrm{precision}@N = \frac{\sum_{k=1}^N rel(k)}{N}$$

Here, $\mathcal{I}_u$ is the set of items adopted by user $u$, and $rel(k)$ is the relevance of a recommendation at position k in the list of recommendations. For ratings, the relevance could be defined as those movies rated above a certain threshold, e.g. $r_{ui}>4.0$. 

**Important to note: since precision is pretty much the same as recall in RS, metrcis usch as the *area under the ROC curve* doesn't have any meaning!!**

<div class = "alert alert-success">
As an example, consider a user that watched the following films:
<br /><br />
'Designated Mourner, The (1997)'
<br />
'Money Talks (1997)'
<br />
'Madame Butterfly (1995)'
<br />
'Batman Forever (1995)'
<br /><br />
The recommended items were: 
<br /><br />
'Batman (1989)' 
<br />
'Madame Butterfly (1995)'
<br /><br />
**What would be the recall and precision @1? and @2?**
<br />
**What do you think of recommending Batman? Is a bad or a good recommendation?**
</div>

Please notice that there isn't any actual difference between precision and recall in the context of RS: both measure the relevance of the recommendations, and tell nothing about items recommended that haven't been adopted by the user. Thus, it make sense to define a normalized recall as:

### $$\mathrm{recall}@N = \frac{\sum_{i=1}^N rel_i}{\mathrm{min}(N, \sum_{i\in \mathcal{I}_u} 1})$$

This way, results are normalized to 1 always.

<div class="alert alert-success">
**Exercise** Implement the above definition of recall
</div>

In [76]:
def recall_at_n(N, test, recommended, train=None):
    """
    :param N: number of recommendations
    :param test: list of movies seen by user in test
    :param train: list of movies seen by user in train. This has to be removed from the recommended list 
    :param recommended: list of movies recommended
    
    :return the recall
    """
    if train is not None: # Remove items in train
        rec_true = []
        for r in recommended:
            if r not in train:
                rec_true.append(r)
    else:
        rec_true = recommended    
    intersection = len(set(rec_true[:N]) & set(test))
    return intersection / float(np.minimum(N, len(test)))

In [77]:
seen = ['Designated Mourner, The (1997)', 'Money Talks (1997)', 
        'Madame Butterfly (1995)', 'Batman Forever (1995)']
recommended = ['Batman (1989)', 'Madame Butterfly (1995)']

In [78]:
recall_at_n(1, seen, recommended)

0.0

In [79]:
recall_at_n(2, seen, recommended)

0.5

In [80]:
# Check it's well normalized
print(recall_at_n(3, seen, recommended))
print(recall_at_n(10, seen, recommended))
print(recall_at_n(100, seen, recommended))

0.3333333333333333
0.25
0.25


### Now, use this implementation to measure the efficiency of the popularity baselines in the test set. Use the top-5 movies, for instance

In [87]:
train_grouped = train.groupby('user_id')['item_id'].apply(list).reset_index()
test_grouped = test.groupby('user_id')['item_id'].apply(list).reset_index()

In [94]:
merged = pd.merge(train_grouped, test_grouped, on='user_id', 
                  how='inner', suffixes=('_train', '_test'))

In [95]:
merged.head()

,user_id,item_id_train,item_id_test
0,1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[289, 290, 373, 374, 665, 447, 844, 1023, 375,..."
1,2,"[8, 14, 36, 80, 105, 218, 202, 206, 219, 220, ...","[289, 5, 9, 18, 90, 196, 207, 453, 384, 620, 6..."
2,3,"[199, 213, 250, 217, 251, 237, 238, 242, 252, ...","[298, 207, 209, 383, 230, 240, 235, 387, 673, ..."
3,4,"[6, 36, 209, 213, 230, 251, 239, 235, 255, 258...","[169, 207, 250, 238, 259, 275]"
4,5,"[289, 290, 12, 17, 291, 28, 48, 292, 51, 53, 6...","[665, 18, 21, 36, 54, 76, 589, 81, 378, 120, 1..."


In [98]:
def func(row, recommendation, N=10):
    train_list = row[1]
    test_list = row[2]
    rec_list = recommendation.id.values
    return recall_at_n(N, test_list, rec_list, train_list)


topn=100
merged.apply(lambda row: func(row, mostRatedMovies, topn), axis=1).mean()

0.42633342524622253

In [102]:
merged.apply(lambda row: func(row, highestMeanRatedMovies, topn), axis=1).mean()

0.3217962459920127

## 3.2 Mean Averaged Precision (MAP) -- Advanced material

Previous metrics did not account for the ranking of the recommendation, i.e. the relative position of a movie within the sorted list of recommendations. **But orders matters!** Metrics like MAP, MRR or NDCG try to tackle down this problem. 

From the blog *http://fastml.com/what-you-wanted-to-know-about-mean-average-precision/*:

> Here’s another way to understand average precision. Wikipedia says AP is used to score document retrieval. You can think of it this way: you type something in Google and it shows you 10 results. It’s probably best if all of them were relevant. If only some are relevant, say five of them, then it’s much better if the relevant ones are shown first. It would be bad if first five were irrelevant and good ones only started from sixth, wouldn’t it? AP score reflects this.

Implementation taken from:

https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py



## Average Precision 

The Average Precision is definied as:

### $$\mathrm{AP}@N = \frac{\sum_{k=1}^N P(k) \times rel(k)}{\mathrm{min}(N, \sum_{i\in \mathcal{I}_u} 1)}$$

where $P(k)$ is the precision at cut-off in the item list, i.e. the ratio of the number of recommended items adopted, up to the position k, over the number k. Thus:

### $$\mathrm{AP}@N = \frac{\sum_{k=1}^N \left(\sum_{i=1}^k rel(i)\right)/k \times rel(k)}{\mathrm{min}(N, \sum_{i\in \mathcal{I}_u} 1)}$$



<div class = "alert alert-success">
Following the example above, consider a user that watched the following films:
<br /><br />
'Designated Mourner, The (1997)'
<br />
'Money Talks (1997)'
<br />
'Madame Butterfly (1995)'
<br />
'Batman Forever (1995)'
<br /><br />
The recommended items were: 
<br /><br />
'Batman (1989)' 
<br />
'Madame Butterfly (1995)'
<br /><br />

<div class = "alert alert-success">
**Calculate AP@1**
<br /><br />
First, *rel(1)=0*, because Batman was not viewed. Also, *P(1) = 0*. Thus, AP@1=0.
<br />
**Calculate AP@2**
<br /><br />
As before, *rel(1)=0*, so the first term does not contribute. For the second term, *rel(2)=1*, so that *P(2)=0.5*. The numerator is hence:
<br /><br />
$P(1)*rel(1)+P(2)*rel(2)=0*0+0.5*1$
<br /><br />
For the denominator, $N=2$ and $\sum_{i\in \mathcal{I}_u} 1)=4$, thus:
<br /><br />
AP@2 = 0.5/2 = 0.25
</div>

Let's now implement it =)

In [ ]:
def apk(N, test, recommended, train=None):
    """
    Computes the average precision at N given recommendations.
    
    :param N: number of recommendations
    :param test: list of movies seen by user in test
    :param train: list of movies seen by user in train. This has to be removed from the recommended list 
    :param recommended: list of movies recommended
    
    :return The average precision at N over the test set
    """
    if train is not None: 
        rec_true = []
        for r in recommended:
            if r not in train:
                rec_true.append(r)
    else:
        rec_true = recommended    
    predicted = rec_true[:N] # top-k predictions
    
    score = 0.0 # This will store the numerator
    num_hits = 0.0 # This will store the sum of rel(i)

    for i,p in enumerate(predicted):
        if p in test and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits/(i+1.0)

    return score / min(len(test), N)

In [ ]:
seen = ['Designated Mourner, The (1997)', 'Money Talks (1997)', 'Madame Butterfly (1995)', 'Batman Forever (1995)']
recommended = ['Madame Butterfly (1995)', 'Batman (1989)']

In [ ]:
apk(1, seen, recommended)

In [ ]:
apk(2, seen, recommended)

In [ ]:
apk(3, seen, recommended)

## MAP

Mean avergae precision is nothing else than the AP averaged across users ;)

Apply it to popularity baselines

In [ ]:
?

<div class="alert alert-success">
The rest of the class is covered in a different notebook
</div>